# Code for simulating disk packings on cones

By Jessica H. Sun and Abigail Plummer

Licensed under the terms of GNU GENERAL PUBLIC LICENSE by Free Software Foundation

Change the simulation_input variable to the name of the desired input file, which can be found in the cone-disk-packings/simulation/simulation_inputs folder. You can generate your own input file if desired using run_dir_list.ipynb.

The simulation will create a folder named /dat in the working directory to contain the outputs.

In [ ]:
simulation_input='run_dir_list_022322_0-99_cylinders0-5cinit.npy'

In [ ]:
import sys
import numpy as np
import random
import os
import zipfile
import io
import pandas as pd
from math import atan2, cos, sin

In [ ]:
if simulation_input.endswith('.zip'):
    inputzip = zipfile.ZipFile('simulation_inputs/'+simulation_input,'r')
    inputzip_bytes = io.BytesIO(inputzip.read(inputzip.infolist()[0]))
    run_dir_list=np.load(inputzip_bytes)
else:
    run_dir_list=np.load('simulation_inputs/'+simulation_input)    

In [ ]:
class ConeBennett:
    def __init__(self, Nmax=500, ctheta=np.pi/8.0, cinit=10,itheta=0):
        self.ctheta=ctheta #sector angle (unwrapped cone angle)
        self.Nmax=Nmax # number of particles that we will place on the surface IN ADDITION to the triangular seed        
        self.a0=1 #distance between particles. For now, we leave it fixed at 1, so all distances are measured in terms of this
        self.itheta=itheta #orientation angle
        self.cinit=cinit #initialization height of seed
        if self.ctheta!=0:
            self.yinit=cinit/self.ctheta #arc length / sector angle = radius of sector
        else: #for a cylinder, assume semi-infinite cone..
            self.yinit=cinit/0.002 
            self.Lx=self.cinit #width of simulation box, which on x-axis is (-Lx/2,+Lx/2]                
        self.tol=0.0001 #define a tolerance for when crystals meet each other again        
        self.n=0 #define the variable we will use to keep track of how many particles we have placed.         
        self.R=self.yinit+20 #height of the cone (radius of circle that the cone is a rolled up sector of)
        self.adj=np.zeros((self.Nmax,self.Nmax)) #define adjacency matrix, which will be symmetric. Each row will represent a particle. in row i, col j is 1 if there exists a bond between i and j and 0 otherwise.
        self.pos=np.zeros((self.Nmax, 2)) #another matrix, N rows, 2 columns. Each row index here is also a row in the adjacency matrix, and we can go back and forth looking up positions and neighbors.
        self.candidates=[] #create a container to hold the candidate states. We will keep track of the location in xy and the energy here.
        self.edgeSites=[] #a list of sites that have open neighbors 
    def saveParticles(self): #save particle positions in x y in the same directory as the code
        fileout='particles.dat'
        outfile=self.pos[:self.n,]    
        np.savetxt(fileout,np.c_[outfile],fmt='%.6g') #you can change the number of decimal places output her 
    def saveMov(self):
        fileout='mov/particles'+str(self.n)+'.dat'
        outfile=self.pos[:self.n,]
        np.savetxt(fileout,np.c_[outfile],fmt='%.6g') #you can change the number of decimal places output here 
    def saveAdj(self):
        fileout='adj.dat'
        outfile=self.adj[:self.n,:self.n]
        np.savetxt(fileout,np.c_[outfile],fmt='%.6g')
    def pbc(self,p0): #moves point into the middle simulation sector
        if self.ctheta!=0:
            angle=atan2(p0[1],p0[0])
            r=np.sqrt(p0[0]**2+p0[1]**2)
            if angle>(-np.pi/2.0+self.ctheta/2.0): #include the RHS in the domain
                angle=angle-np.floor((abs(angle-(-np.pi/2))+self.ctheta*0.5)/self.ctheta)*self.ctheta
            if angle<=(-np.pi/2.0-self.ctheta/2.0):
                angle=angle+np.floor((abs(angle-(-np.pi/2))+self.ctheta*0.5)/self.ctheta)*self.ctheta
            return [r*cos(angle),r*sin(angle)]
        else: #if cylinder...
            x,y=p0[0],p0[1]
            if x>self.Lx/2.0:
                x=x-np.floor((x+self.Lx*0.5)/(self.Lx))*self.Lx
            if x<=-self.Lx/2.0:
                x=x+np.floor(abs(x-self.Lx*0.5)/self.Lx)*self.Lx
            return [x,y]            
    def periodicImages(self,x1,y1): #moves point in the niddle simulation sector into L & R image positions
        if self.ctheta!=0:
            angle=atan2(y1,x1)
            r=np.sqrt(x1**2+y1**2)
            angleR=angle+self.ctheta #move to the image sector to the right
            angleL=angle-self.ctheta #move to image sector to the left
            return [r*cos(angleR), r*sin(angleR), r*cos(angleL), r*sin(angleL)] #output a list of the positions of the right image and left image points
        else:
            x,y=x1,y1 #move to the image sector to the right
            xR=x1+self.Lx #move to image sector to the left
            xL=x1-self.Lx #output a list of the positions of the right image and left image points
            return [xR,y1,xL,y1]    
    def findDistance(self,p0, p1): #we need something that returns the minimum distance between two particles, respecting periodic boundaries.
        pIm=self.periodicImages(p1[0],p1[1])
        d1=np.sqrt((p0[0]-p1[0])**2+(p0[1]-p1[1])**2)
        d2=np.sqrt((p0[0]-pIm[0])**2+(p0[1]-pIm[1])**2)
        d3=np.sqrt((p0[0]-pIm[2])**2+(p0[1]-pIm[3])**2)
        return min([d1,d2,d3])
    def findDistanceandIm(self,p0, p1): #also return the image point used to compute distances
        pIm=self.periodicImages(p1[0],p1[1])
        d1=np.sqrt((p0[0]-p1[0])**2+(p0[1]-p1[1])**2)
        d2=np.sqrt((p0[0]-pIm[0])**2+(p0[1]-pIm[1])**2)
        d3=np.sqrt((p0[0]-pIm[2])**2+(p0[1]-pIm[3])**2)
        if d1<=d2 and d1<=d3:
            return [d1, p1[0], p1[1]]
        if d2<=d1 and d2<=d3:
            return [d2, pIm[0], pIm[1]]
        if d3<=d1 and d3<=d2:
            return [d3, pIm[2], pIm[3]]
    def boundary(self,pos): #return a bool that says whether or not we are within the boundary, given an i,j point
        if np.sqrt(pos[0]**2+pos[1]**2)<self.R: 
            return True
        else:
            return False    
    def findMutualSites(self, dist, x0,y0,x1,y1): #this method takes two pre-existing points and finds two other points that are a distance a0 from BOTH of them. Only works when the distance between the two points is between 1 and 2
        midpt=[(x0+x1)/2.0, (y0+y1)/2.0]
        perpbisector=[(y1-y0), -(x1-x0)]
        altitude=np.sqrt(self.a0**2-(dist/2.0)**2)
        pos1=self.pbc([midpt[0]+perpbisector[0]/(1.0*dist)*altitude, midpt[1]+perpbisector[1]/(1.0*dist)*altitude])
        pos2=self.pbc([midpt[0]-perpbisector[0]/(1.0*dist)*altitude, midpt[1]-perpbisector[1]/(1.0*dist)*altitude])
        return [pos1, pos2]
    def noPreexistingPart(self,l, m, pos1, pos2): #consider the two points that connect to our candidate state-- is one of them already 'filled'?        
        pos1Bool=True #define two bools, and they will be true if there is no overlap from a preexisting neighbor, and false otherwise
        pos2Bool=True	        
        mutuals=self.adj[l,]+self.adj[m,] #we sum up adjacency matrix to find neighbors that they both share. Anywhere that there is a 2, it is a mutual neighbor. 
        mutualIdx=(np.where(mutuals==2))[0] #get a np array of anywhere that it sums to two 
        for j in mutualIdx:
            neighbpos=self.pos[mutualIdx[0],]
            if abs(self.findDistance(pos1, neighbpos))<0.001: #like in noDuplicates, we hard code in a value instead of using tol bc this is asking the question-- does this EXACT point already exist? 
                pos1Bool=False
            if len(pos2)>0:
                if abs(self.findDistance(pos2, neighbpos))<0.001:
                    pos2Bool=False
        return pos1Bool, pos2Bool    
    def findEnergy(self, candpos): #findEnergy now returns a bondlist, so that we can easily update the adjacency matrix. 
        en=0
        bondlist=[]
        for idx in self.edgeSites: #note that the candidate state is not part of edgeSites, so shouldn't have redundancy. The two neighbors that we used to generate candpos SHOULD be in edgeSites. 
            dist=self.findDistance(candpos, self.pos[idx,])
            if abs(dist-self.a0)<self.tol:
                en=en-1
                bondlist.append(idx)
            if dist-self.a0<(-1*self.tol): #energy penalty if it overlaps
                en=en+100
        return en, bondlist
    def noDuplicates(self, candpos, bondlist): #returns true if candidatepos with bondlist is not already in self.candidates list. Only add it if it's not already there. 
        duplicateBool=True
        for j in self.candidates:
            if set(bondlist)==set(j[2]):                
                if abs(self.findDistance(candpos, j[0]))<0.001: #we don't use tol here, because this is not about overlap-- we want to remove duplicate candidates
                    duplicateBool=False
        return duplicateBool
    def genCandidatesforPair(self, l, m):
        dist, x1, y1=self.findDistanceandIm(self.pos[l], self.pos[m])
        x0,y0=self.pos[l]
        if abs(dist-2*self.a0)<self.tol: #if the base pair particles are exactly 2 apart, the only candidate lies directly in between them. average the points, pbc wrapper
            candpos=self.pbc([(x0+x1)/2.0, (y0+y1)/2.0])
            if (self.noPreexistingPart(l,m,candpos, [])[0]) and self.boundary(candpos):
                en, bondlist=self.findEnergy(candpos)
                if en<-1 and self.noDuplicates(candpos, bondlist):
                    self.candidates.append([candpos, en, bondlist])
        if abs(dist-self.a0)<self.tol or (dist>self.a0 and dist<(2*self.a0) and abs(dist-2*self.a0)>self.tol): #if the pair is exactly 1 apart, we have a crystalline-like packing problem, and we can check neighbors to speed up computation. if it is between 1 and 2 apart, we still generate new candidates by solving for the vertices of a rhombus.
            candpos=self.findMutualSites(dist,x0,y0,x1,y1)
            boolList=self.noPreexistingPart(l,m,candpos[0],candpos[1])	
            for idx,val in enumerate(boolList):
                if val and self.boundary(candpos[idx]):
                    en, bondlist=self.findEnergy(candpos[idx])
                    if en<-1 and self.noDuplicates(candpos[idx], bondlist):
                        self.candidates.append([candpos[idx], en, bondlist])
    def analyzeInitSeed(self): #append the initial seed to edgeSites(assumes that we don't initialize with any instances of 6 fold coordination, but even if we do, nothing major goes wrong)    
        self.edgeSites=list(range(self.n))
        for j in range(self.n):
            for k in range((j+1), self.n):
                dist=self.findDistance(self.pos[j], self.pos[k])
                if abs(dist-self.a0)<self.tol:
                    self.adj[j,k]=self.adj[k,j]=1
        for l in range(self.n):
            for m in range((l+1), self.n):
                self.genCandidatesforPair(l,m)
    def rotate(self, point, theta):
        x=point[0]
        y=point[1]
        rot_point=[x*np.cos(theta)-y*np.sin(theta),x*np.sin(theta)+y*np.cos(theta)]
        return rot_point
    def initialize(self): #initialize system with whatever the desired pattern is.
        ninit=3 # initialize triangular seed
        triangle0=np.array([0,0]) #place initial points in triangle with one point on origin at all times
        triangle1=np.array([1,0])
        triangle2=np.array([0.5,np.sqrt(3)/2.0])
        rot_triangle0=self.rotate(triangle0,self.itheta)
        rot_triangle1=self.rotate(triangle1,self.itheta)
        rot_triangle2=self.rotate(triangle2,self.itheta)        
        self.pos[0,]=[rot_triangle0[0], rot_triangle0[1]-self.yinit]
        self.pos[1,]=[rot_triangle1[0],rot_triangle1[1]-self.yinit]
        self.pos[2,]=[rot_triangle2[0], rot_triangle2[1]-self.yinit]
        self.n=self.n+ninit #increment n by the number of particles that you just added. When n=Nmax, this is done.         
        for i in range(self.n): #confirm that initial seed is within the domain
            if not self.boundary(self.pos[i]):
                print('initial seed is outside of domain')
        self.analyzeInitSeed() #generate the initial candidate list, based on the starting seed, and fill in the adjacency matrix        
        # random.seed(10) #seed the random number generator so it is 'deterministically random' data. Comment out if you want 'truly' random. 
    def selectNewSite(self): #returns an index OF THE CANDIDATES LIST that we are going to choose
        if len(self.candidates)==0: #if we have no remaining candidates, throw an error. 
            self.saveParticles()
            self.saveAdj()
            raise Exception('no acceptable candidates')        
        envalues=[x[1] for x in self.candidates] #make a list of the energies
        minval=min(envalues)        
        indices = [idx for idx, x in enumerate(envalues) if x == minval] #find all places where the minimum is-- indices is a list of indices
        if len(indices)>1:
            sel=random.choice(indices)
        if len(indices)==1:
            sel=indices[0]
        return sel

    def reCalcCandEnergy(self): # need to make a dummy list to keep track of who doesn't overlap with our new particle wedged in there. Otherwise, we'd have particles that were ~100 energy in candidates list    
        stillAccessible=[]
        for idx, val in enumerate(self.candidates):
            self.candidates[idx][1], self.candidates[idx][2]=self.findEnergy(self.candidates[idx][0])
            #if no overlap was created, we keep this as a candidate.
            if self.candidates[idx][1]<-1:
                stillAccessible.append(self.candidates[idx])
        self.candidates=stillAccessible
    def addParticle(self): #find the index of the candidate that we want to place a particle 
        sel=self.selectNewSite()
        newpos, en, bondlist=self.candidates[sel]        
        del self.candidates[sel] #remove this particle from the candidate list- we cannot pick it again        
        self.pos[self.n]=newpos #add to the position list        
        for j in bondlist: #add the appropriate bonds in bondlist
            self.adj[self.n,j]=self.adj[j,self.n]=1
        self.edgeSites.append(self.n)        
        self.reCalcCandEnergy() #recalculate the energy of the candidate states (in case they were affected by new particle addition). So have to append n to endsites first here.        
        for j in self.edgeSites[:-1]: #generate new candidates. genCandidatesforPair screens for duplicates itself.
            self.genCandidatesforPair(j, self.n)        
        self.n=self.n+1 #now that we have fully 'added the particle' we increment n (this relative ordering of steps keeps the indexing right.)    
    def run(self): #pull it all together!
        try:
            self.initialize()
            while self.n<self.Nmax:
                try:
                    self.addParticle()
                    # self.saveMov()
                except: #likely Nmax > dense packing n so break when no more particles can be placed
                    print('break at n='+str(self.n),end=' | ')
                    break
            self.saveParticles()
            self.saveAdj()
        except:
            print('error')

In [ ]:
def get_params_dict(run_dir):
    keys=[]
    values=[]
    for params in run_dir.split('_'):
        key,value=params.split('-')
        keys+=[key]
        values+=[value]
    params_dict=dict(zip(keys,values))
    return params_dict

output_dir=os.getcwd()+'/datfiles'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

counter=0
total=len(run_dir_list)
for run_dir in run_dir_list:
    counter+=1
    os.chdir(output_dir)
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    if not (os.path.exists(run_dir+'/particles.dat'))&(os.path.exists(run_dir+'/adj.dat')): #in the future, edit this to check to make sure adj and particles aren't empty (indicates file writing was interrupted)
        os.chdir(output_dir+'/'+run_dir)
        params_dict=get_params_dict(run_dir)
        
        model=ConeBennett(int(float(params_dict['Nmax'])),float(params_dict['cthetadeg'])/180*np.pi,float(params_dict['cinit']),float(params_dict['ithetadeg'])/180*np.pi)
        model.run()
        print(run_dir+' done! ('+str(counter)+' / '+str(total)+')')